# Data science Capstone project -- Segmenting and cluster Toronto Neighborhood

## 1. Import Libraries and Read From Wiki Page

In [1]:
import pandas as pd
import numpy as np

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
tables = pd.read_html(url,header=0)

In [3]:
neighbor_df = tables[0]
neighbor_df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## 2. Data Clean Up and Wrangling

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [4]:
neighbor_df.columns=['PostalCode','Borough','Neighborhood']

In [5]:
neighbor_df.shape

(288, 3)

In [6]:
neighbor_df[neighbor_df.Borough=='Not assigned'].shape

(77, 3)

In [7]:
neighbor_df = neighbor_df[neighbor_df.Borough!='Not assigned']

In [8]:
neighbor_df.shape

(211, 3)

More than one neighborhood can exist in one postal code area. For example, M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [9]:
neighbor_df[neighbor_df.PostalCode=='M5A']

,PostalCode,Borough,Neighborhood
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


In [10]:
neighbor = neighbor_df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
neighbor.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [11]:
neighbor.loc[neighbor.Neighborhood=='Not assigned']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [12]:
neighbor.loc[neighbor.Neighborhood=='Not assigned','Neighborhood'] = neighbor.loc[neighbor.Neighborhood=='','Borough']

In [13]:
neighbor[neighbor.Neighborhood=='Not assigned']

,PostalCode,Borough,Neighborhood


In [14]:
neighbor.shape

(103, 3)

## 3. Load Neiborhood Location Geocode

In [15]:
geocode_df = pd.read_csv('Geospatial_Coordinates.csv')
geocode_df.columns=['PostalCode','Latitude','Longitude']
geocode_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
neighbor_geocode = pd.merge(neighbor, geocode_df, on='PostalCode')
print(neighbor_geocode.shape)
neighbor_geocode.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## 4. Explore and cluster the neighborhoods in Toronto

We will limit the analysis to boroughs that contains the word "Toronto"

First we initialize our Foursqure API ID details

In [17]:
CLIENT_ID = 'UZSXPNV5Z24ZYOEFAH1RLCZ1VDTC2O44NRKZOBUB01GHEJIL' # your Foursquare ID
CLIENT_SECRET = 'SJCCMG3AM21ON0KAZSITFRFGM2WBOXTNGN2MMAOZVKEK4EGU' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: UZSXPNV5Z24ZYOEFAH1RLCZ1VDTC2O44NRKZOBUB01GHEJIL
CLIENT_SECRET:SJCCMG3AM21ON0KAZSITFRFGM2WBOXTNGN2MMAOZVKEK4EGU


Second, we limit the dataframe to include only the boroughs that contain the word "Toronto"

In [18]:
mask = neighbor_geocode.Borough.str.contains("Toronto")
mask.head()

0    False
1    False
2    False
3    False
4    False
Name: Borough, dtype: bool

In [20]:
toronto_df = neighbor_geocode[mask]
print("Great Toronto Boroughs have {} boroughs and {} neighborhoods".format(len(toronto_df['Borough'].unique()), toronto_df.shape[0]))

Great Toronto Boroughs have 4 boroughs and 38 neighborhoods


## 5. Create a map of Toronto with neighborhoods superimposed on top

In [21]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [22]:
address = 'Toronto, CANADA'

geolocator = Nominatim(user_agent="tt_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [23]:
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, postcode in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['PostalCode']):
    label ='{}, {}'.format(borough, postcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius=5,
        popup =label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
    
map_toronto

## 6. Explore Neighborhoods in Toronto

In [24]:
def getNearbyVenues(names, latitudes, longitudes, radius=50):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
toronto_df['Neighborhood']
toronto_venues=getNearbyVenues(toronto_df['Neighborhood'], toronto_df['Latitude'], toronto_df['Longitude'], radius=500)

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [27]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


#### Let's check how many venues in each Borough

In [28]:
toronto_venues.groupby(['Neighborhood'])['Neighborhood'].count()

Neighborhood
Adelaide, King, Richmond                                                                                      30
Berczy Park                                                                                                   30
Brockton, Exhibition Place, Parkdale Village                                                                  21
Business Reply Mail Processing Centre 969 Eastern                                                             19
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara    14
Cabbagetown, St. James Town                                                                                   30
Central Bay Street                                                                                            30
Chinatown, Grange Park, Kensington Market                                                                     30
Christie                                                                           

#### Let's find out how many unique categories from those neighborhoods

In [29]:
len(toronto_venues['Venue Category'].unique())

191

## 7. Analyze Each Neighborhood

We convert categorical data column **Venue Category** into numbers, using pd.get_dummies

In [63]:
col = list(toronto_onehot.columns)
print(len(col))
col.remove("Neighborhood")
print(len(col))

191
190


In [66]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
print(toronto_onehot.shape)

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
print(toronto_onehot.shape)

# Move column Neighborhood to the first place
col = list(toronto_onehot.columns)
col.remove("Neighborhood")
columns=['Neighborhood'] + col
toronto_onehot = toronto_onehot[columns]
toronto_onehot.head()

(825, 191)
(825, 191)


,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"The Danforth West, Riverdale",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
toronto_onehot.describe()

,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
count,825.000000,825.000000,825.000000,825.000000,825.000000,825.000000,825.000000,825.000000,825.000000,825.000000,...,825.000000,825.000000,825.000000,825.000000,825.000000,825.000000,825.000000,825.000000,825.000000,825.000000
mean,0.001212,0.001212,0.001212,0.001212,0.002424,0.002424,0.002424,0.012121,0.001212,0.002424,...,0.001212,0.001212,0.004848,0.001212,0.010909,0.001212,0.007273,0.002424,0.001212,0.008485
std,0.034816,0.034816,0.034816,0.034816,0.049207,0.049207,0.049207,0.109493,0.034816,0.049207,...,0.034816,0.034816,0.069504,0.034816,0.103938,0.034816,0.085021,0.049207,0.034816,0.091777
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [69]:
toronto_grouped =toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

(38, 191)

In [70]:
toronto_grouped.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.0,...,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.047619
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.052632
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


#### Let's print each neighborhood along with the top 5 most common venues

In [89]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    #print(row_categories_sorted)
    #print(row_categories_sorted.index.values[0:num_top_venues])
    return row_categories_sorted.index.values[0:num_top_venues]

In [88]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",Steakhouse,Café,Pizza Place,Asian Restaurant,Hotel
1,Berczy Park,Seafood Restaurant,Cocktail Bar,Beer Bar,Farmers Market,Café
2,"Brockton, Exhibition Place, Parkdale Village",Breakfast Spot,Coffee Shop,Café,Climbing Gym,Burrito Place
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Airport


## 8. Cluster the Neighborhood

Run *k*-means to cluster the neighborhood into 5 clusters

In [93]:
from sklearn.cluster import KMeans

kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', axis=1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)


kmeans.labels_[0:10]
#toronto_grouped= pd.merge(toronto_df,Toronto_grouped, on='Neighborhood', how='right')

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [94]:
toronto_grouped_clustering.head()

,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.033333,0.0,0.0,...,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.033333,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.047619
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.052632
4,0.0,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000


Create a new dataframe that includes the cluster as well as top 10 venues from each neighborhood.

In [108]:
neighborhoods_venues_sorted.head()

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Adelaide, King, Richmond",0,Steakhouse,Café,Pizza Place,Asian Restaurant,Hotel
1,Berczy Park,0,Seafood Restaurant,Cocktail Bar,Beer Bar,Farmers Market,Café
2,"Brockton, Exhibition Place, Parkdale Village",0,Breakfast Spot,Coffee Shop,Café,Climbing Gym,Burrito Place
3,Business Reply Mail Processing Centre 969 Eastern,0,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Airport


In [132]:
#neighborhoods_venues_sorted.insert(1, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

toronto_merged = neighbor_geocode
toronto = neighborhoods_venues_sorted.join(toronto_merged.set_index("Neighborhood"),on='Neighborhood')
toronto.head()

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,PostalCode,Borough,Latitude,Longitude
0,"Adelaide, King, Richmond",0,Steakhouse,Café,Pizza Place,Asian Restaurant,Hotel,M5H,Downtown Toronto,43.650571,-79.384568
1,Berczy Park,0,Seafood Restaurant,Cocktail Bar,Beer Bar,Farmers Market,Café,M5E,Downtown Toronto,43.644771,-79.373306
2,"Brockton, Exhibition Place, Parkdale Village",0,Breakfast Spot,Coffee Shop,Café,Climbing Gym,Burrito Place,M6K,West Toronto,43.636847,-79.428191
3,Business Reply Mail Processing Centre 969 Eastern,0,Yoga Studio,Auto Workshop,Park,Comic Shop,Pizza Place,M7Y,East Toronto,43.662744,-79.321558
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Airport,M5V,Downtown Toronto,43.628947,-79.394420


Now we show neighborhood clustering on a map

In [133]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [134]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)


x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]


markers_colors = []
for lat, lon, poi, cluster in zip(toronto['Latitude'], toronto['Longitude'], toronto['Neighborhood'], toronto['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters